<a href="https://colab.research.google.com/github/DiGyt/neuropynamics/blob/lucas_dev/notebooks/Single_neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparations

## Installation of required packages

In [1]:
# We use the brian2 toolbox for our models in the backend
!pip install brian2

     |████████████████████████████████| 1.6MB 6.1MB/s 
     |████████████████████████████████| 5.8MB 38.3MB/s 
  Created wheel for brian2: filename=Brian2-2.3.0.2-cp36-cp36m-linux_x86_64.whl size=1283464 sha256=1eb8225ce677be5867772f6c347e616a1ce11d7583255b9c1ec61d618fbbdb60
  Stored in directory: /root/.cache/pip/wheels/86/b5/f6/0c8f9eec58c01e4af55a4c79a5e0385aea01ed03e4ec36c4ac
Successfully built brian2
  Found existing installation: sympy 1.1.1
    Uninstalling sympy-1.1.1:
      Successfully uninstalled sympy-1.1.1


## Cloning of GitHub repo

This enables us to access all the models and useful code we created for our toolbox in a Colab notebook

In [2]:
!git clone https://github.com/DiGyt/neuropynamics/ --branch lucas_dev

Cloning into 'neuropynamics'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 91 (delta 26), reused 36 (delta 1), pack-reused 0
Unpacking objects: 100% (91/91), done.



## Imports

In [3]:
# Main packages
import numpy as np

# Brian2 package
# Unit definitions
from brian2 import mV, ms, volt, second, umetre, ufarad, siemens, cm, msiemens, amp, uA, nA
# Other stuff
from brian2 import start_scope, NeuronGroup, StateMonitor, run

# Plotting stuff
import matplotlib.pyplot as plt
import seaborn as sns
from neuropynamics.src.utils.plotting import create_default_plot

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, HBox, VBox, Layout

/usr/local/lib/python3.6/dist-packages/setuptools/distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "
WARNING    /usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
 [py.warnings]


# Simulations of single neurons

## Izhikevich model

In [ ]:
# Import the wrapper function to create a Izhikevich neuron easily in brian2 
from neuropynamics.src.models.neurons import create_izhikevich_neuron

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_izhikevich_neuron(I_ext, a, b, c, d):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_izhikevich_neuron(Vmax = 35)      
  # Set neuron parameters
  a = a/ms
  b = b/ms
  c = c * mV 
  d = d * volt/second  
  # Start monitoring the neurons state
  statemon = StateMonitor(neuron, 'vm', record=0)
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Set input current to neuron
  neuron.I = I_ext * volt / second
  # Run 500ms with input
  run(500*ms)
  # Remove input current to neuron
  neuron.I = 0 * volt / second
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Plot results
  create_default_plot(x = statemon.t/ms, y = statemon.vm[0], title = 'Izhikevich Neuron', x_label = 'Time (ms)', y_label = 'Membrane Potential (mV)')

# Set default parameters 
I_ext_def = 10.
a_def = 0.02
b_def = 0.2
c_def = -65.
d_def = 8.

# Create sliders for neuron parameters
a_slider = widgets.FloatSlider(value = a_def, min = 0., max = 0.15, step = 0.01, description = 'a:', readout_format = '.2f')
b_slider = widgets.FloatSlider(value = b_def, min = 0., max = 0.35, step = 0.01, description = 'b:', readout_format = '.2f') # Somehow max = 0.3 does not work
c_slider = widgets.FloatSlider(value = c_def, min = -75., max = -40., step = 1, description = 'c:', readout_format = '.1f')
d_slider = widgets.FloatSlider(value = d_def, min = 0., max = 10., step = 1, description = 'd:', readout_format = '.1f')

# Create slider for input current
I_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 40., step = 1, description = 'I:', readout_format = '.1f')

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_izhikevich_neuron, I_ext = I_slider, a = a_slider, b = b_slider, c = c_slider, d = d_slider, Vmax = fixed(35))

# Create functions to set specific neuron configurations
def apply_config(I_ext, a, b, c, d):
  I_slider.value = I_ext
  a_slider.value = a
  b_slider.value = b
  c_slider.value = c
  d_slider.value = d

def reset_config(name):
  apply_config(I_ext = I_ext_def, a = a_def, b = b_def, c = c_def, d = d_def)

def apply_intrinsically_bursting_config(name):
  apply_config(I_ext = 12., a = a_def, b = b_def, c = -55, d = 4)

def apply_chattering_config(name):
  apply_config(I_ext = I_ext_def, a = a_def, b = b_def, c = -50, d = 2)

def apply_fast_spiking_config(name):
  apply_config(I_ext = I_ext_def, a = 0.1, b = b_def, c = c_def, d = d_def)

def apply_low_thresh_spiking_config(name):
  apply_config(I_ext = I_ext_def, a = a_def, b = 0.25, c = c_def, d = d_def)

def apply_resonator_config(name):
  apply_config(I_ext = I_ext_def, a = 0.1, b = 0.26, c = c_def, d = d_def)

# Create buttons for specific neuron configurations
# Set button layout
button_layout = Layout(width='180px', height='30px')
# Reset button
regular_spiking_button = widgets.Button(description='Regular Spiking', layout = button_layout)
regular_spiking_button.on_click(reset_config)
# Intrinsically bursting
intrinsically_bursting_button = widgets.Button(description='Intrinsically Bursting', layout = button_layout)
intrinsically_bursting_button.on_click(apply_intrinsically_bursting_config)
# Chattering neuron
chattering_button = widgets.Button(description='Chattering', layout = button_layout)
chattering_button.on_click(apply_chattering_config)
# Fast spiking
fast_spiking_button = widgets.Button(description='Fast Spiking', layout = button_layout)
fast_spiking_button.on_click(apply_fast_spiking_config)
# Low-threshold spiking
low_thresh_spiking_button = widgets.Button(description='Low-threshold Spiking', layout = button_layout)
low_thresh_spiking_button.on_click(apply_low_thresh_spiking_config)
# Resonator
resonator_button = widgets.Button(description='Resonator', layout = button_layout)
resonator_button.on_click(apply_resonator_config)

# Place buttons into grid
button_description = widgets.Label(value='Select predefined neuron types:', layout = Layout(width='300px', height='30px'))
buttons = HBox(children=[regular_spiking_button, intrinsically_bursting_button, chattering_button, fast_spiking_button, low_thresh_spiking_button, resonator_button])

# Display main widget and buttons
parameter_description = widgets.Label(value='Set parameters yourself:', layout = Layout(width='300px', height='30px'))
display(VBox(children=[VBox(children=[button_description, buttons]), VBox(children=[parameter_description, main_widgets])]))

## Hodgkin-Huxley model

In [31]:
# Import the wrapper function to create a Izhikevich neuron easily in brian2 
from neuropynamics.src.models.neurons import create_hodgkin_huxley_neuron

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_hh_neuron(I_ext, g_leak, g_k, g_na, e_leak, e_k, e_na, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_hodgkin_huxley_neuron(Vmax = Vmax)      
  # Set neuron parameters
  area = 20000*umetre**2
  Cm = 1*ufarad*cm**-2 * area # Mebrane capacitance
  gl = g_leak*siemens*cm**-2 * area
  El = e_leak*mV
  EK = e_k*mV
  ENa = e_na*mV
  g_na = g_na*msiemens*cm**-2 * area
  g_kd = g_k*msiemens*cm**-2 * area
  VT = -63*mV
  # Set initial voltage to leak conductance
  neuron.v = El  
  # Start monitoring the neurons state
  statemon = StateMonitor(neuron, 'v', record=0)
  # Run neuron simulation for 10ms without input
  run(10*ms)
  # Set input current to neuron
  neuron.I = I_ext * nA
  # Run 100ms with input
  run(100*ms)
  # Remove input current to neuron
  neuron.I = 0 * nA
  # Run neuron simulation for 10ms without input
  run(10*ms)
  # Plot results
  create_default_plot(x = statemon.t/ms, y = statemon.v[0]/mV, title = 'Hodgkin-Huxley Neuron', x_label = 'Time (ms)', y_label = 'Membrane Potential (mV)')

# Set default parameters 
I_ext_def = 0.3
g_leak_def = 5e-5
g_k_def = 30.
g_na_def = 100.
e_leak_def = -65
e_k_def = -90.
e_na_def = 50.

# Create sliders for neuron parameters
g_leak_slider = widgets.FloatSlider(value = g_leak_def, min = 0., max = 10e-5, step = 1e-6, description = 'g_leak:', readout_format = '.5f')
g_k_slider = widgets.FloatSlider(value = g_k_def, min = 0., max = 70., step = 1, description = 'g_k:', readout_format = '.1f')
g_na_slider = widgets.FloatSlider(value = g_na_def, min = 80., max = 160., step = 1, description = 'g_na:', readout_format = '.1f')
e_leak_slider = widgets.FloatSlider(value = e_leak_def, min = -70., max = -40., step = 1, description = 'E_leak:', readout_format = '.1f')
e_k_slider = widgets.FloatSlider(value = e_k_def, min = -100., max = -50., step = 1, description = 'E_k:', readout_format = '.1f')
e_na_slider = widgets.FloatSlider(value = e_na_def, min = 20., max = 80., step = 1, description = 'E_na:', readout_format = '.1f')

# Create slider for input current
I_ext_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 1., step = 0.01, description = 'I:', readout_format = '.2f')

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_hh_neuron, I_ext = I_ext_slider, g_leak = g_leak_slider, g_k = g_k_slider, g_na = g_na_slider, e_leak = e_leak_slider, e_k = e_k_slider, e_na = e_na_slider, Vmax = fixed(-40))

# Display main widgets
display(main_widgets)

interactive(children=(FloatSlider(value=0.3, description='I:', max=1.0, step=0.01), FloatSlider(value=5e-05, d…